# Inverse Material Renderer
### CS 77/277: Computer Graphics
### Jessie Li & Michael Riad Zaky

In [ ]:
from bsdf import NUM_VAR_BSDF, NUM_VAR_DIFFUSE
import json
import numpy as np
from PIL import Image
import texelSolve
import time
from concurrent.futures import ProcessPoolExecutor

# data
DATA_NAME = 'cerberus512-8scenes'
DATA_FILE = f'./data/{DATA_NAME}.json'

# optimization
NUM_PROCESSES = 8
PROCESS_TIMEOUT = 3000

# parameters for scipy.optimize.minimize
OPTIMIZER_PARAMETERS = {
    'w0': [0.01] * 5,
    'bounds': [(0, 1)] * 5,
    'method': 'Nelder-Mead',
    'methodOptions': { 'xatol': 1/256, 'maxiter': 100, 'adaptive': True },
    'clampColors': True,
    'minScenes': 4,
}

GAMMA = 2.2

# results
OUT_DIRECTORY = f'out/{DATA_NAME}'
OUT_TXT = f'{OUT_DIRECTORY}/result.txt'

IMAGE_PREDICTIONS_RGB = f'{OUT_DIRECTORY}/rgb.png'
IMAGE_PREDICTIONS_ROUGHNESS = f'{OUT_DIRECTORY}/rough.png'
IMAGE_PREDICTIONS_METALLIC = f'{OUT_DIRECTORY}/metal.png'
IMAGE_ERRORS = f'{OUT_DIRECTORY}/errors.png'

IMAGE_RGB_GAMMA = f'{OUT_DIRECTORY}/rgb-gamma-22.png'
IMAGE_ROUGH_GAMMA = f'{OUT_DIRECTORY}/rough-gamma-22.png'
IMAGE_METAL_GAMMA = f'{OUT_DIRECTORY}/metal-gamma-22.png'

In [2]:
def loadJsonData(filePath: str):
    try:
        with open(filePath) as file:
            data = json.load(file)
    except Exception as e:
        print('Error loading JSON:', e)
        return
    
    bsdf = data.get('bsdf', '(null)')
    resolution = data.get('solveTexSize', '(null)')
    
    print(f'File: {filePath}')
    print(f'BSDF: {bsdf}')
    print (f'Resolution: {resolution}')
    print('------------------------------------------------------------')

    return bsdf, resolution, data['column_row_scene']

# -----------------------------------------------------------------
# load data
# -----------------------------------------------------------------
bsdf, resolution, data = loadJsonData(DATA_FILE)
print('\nData loaded.\n')

File: ./data/cerberus512-8scenes.json
BSDF: bsdf
Resolution: [512, 512]
------------------------------------------------------------

Data loaded.



In [31]:
# -----------------------------------------------------------------
# solve, multiprocessing with ProcessPoolExecuter (manual chunks)
# -----------------------------------------------------------------

chunkSize = len(data) // NUM_PROCESSES
dataItems = list(data.items())

errors = np.zeros(resolution) 
predictions = np.zeros((*resolution, NUM_VAR_BSDF))

print(f'Creating {NUM_PROCESSES} processes with chunk size {chunkSize}')

# ProcessPoolExecutor usage:
# stackoverflow.com/questions/75838200/whats-the-simple-way-to-get-the-return-value-of-a-function-passed-to-multiproce
with ProcessPoolExecutor() as executor:
    start_time = time.time()

    futures = []
    
    for i in range(NUM_PROCESSES):
        chunk = dataItems[i * chunkSize : (i + 1) * chunkSize]
        futures.append(executor.submit(texelSolve.processChunk, chunk, bsdf, resolution, OPTIMIZER_PARAMETERS))
    
    for future in futures:
        try:
            chunkPredictions, chunkErrors = future.result(timeout=PROCESS_TIMEOUT)
            
            # update predictions
            mask = chunkPredictions != 0
            predictions[mask] = chunkPredictions[mask]  
            print('Updated predictions.')
            
            # update errors
            mask = chunkErrors != 0
            errors[mask] = chunkErrors[mask]
            print('Updated errors.')

        except TimeoutError:
            print("Timed out.")
        except Exception as e:
            print(f"Error: {e}")
    
    end_time = time.time()
    print(f"\nTime elapsed ({NUM_PROCESSES} processes): {end_time - start_time:.6f} seconds")
    print(f'Nonzero count after solve (should be >> 0): {np.count_nonzero(predictions)}')

Creating 8 processes with chunk size 63
Updated predictions.
Updated errors.
Updated predictions.
Updated errors.
Updated predictions.
Updated errors.
Updated predictions.
Updated errors.
Updated predictions.
Updated errors.
Updated predictions.
Updated errors.
Updated predictions.
Updated errors.
Updated predictions.
Updated errors.

Time elapsed (8 processes): 218.790192 seconds
Nonzero count after solve (should be >> 0): 405746


In [ ]:
with open(OUT_TXT, 'w') as f:
    f.write(f'Creating {NUM_PROCESSES} processes with chunk size {chunkSize}\n\n')
    f.write(f'Time elapsed ({NUM_PROCESSES} processes): {end_time - start_time:.6f} seconds\n')
    f.write(f'Nonzero count after solve (should be >> 0): {np.count_nonzero(predictions)}\n\n')
    
    f.write(f'Max error: {errors.max()}\n')
    f.write(f'Mean error (including zeros): {errors.mean()}\n')
    f.write(f'Mean error (excluding zeros): {np.mean(errors[np.nonzero(errors)])}\n\n')
    
print(f'Max error: {errors.max()}')
print(f'Mean error (including zeros): {errors.mean()}')
print(f'Mean error (excluding zeros): {np.mean(errors[np.nonzero(errors)])}')

In [3]:
# -----------------------------------------------------------------
# solve, multiprocessing on each row (instead of chunk of rows)
# -----------------------------------------------------------------
from multiprocessing import Pool

pendingResults = []
errors = np.zeros(resolution) 
predictions = np.zeros((*resolution, NUM_VAR_BSDF))

print(f'Creating pool with {NUM_PROCESSES} workers, operating on rows')

with Pool(processes=NUM_PROCESSES) as pool: 
    start_time = time.time()

    count = 0
    for v, row in data.items():
        count += 1
        result = pool.apply_async(texelSolve.processRow, args=(v, row, bsdf, resolution[0], OPTIMIZER_PARAMETERS))
        pendingResults.append(result)
    
    pool.close()
    pool.join()
    
    for result in pendingResults:
        v, rowPredictions, rowErrors = result.get(timeout=60)
        
        # update predictions
        predictions[v] = rowPredictions 
        # print('Updated predictions.')
        
        # update errors
        errors[v] = rowErrors
        # print('Updated errors.')

end_time = time.time()
print(f"\nTime elapsed ({NUM_PROCESSES} processes): {end_time - start_time:.6f} seconds")
print(f'Nonzero count after solve (should be >> 0): {np.count_nonzero(predictions)}')

Creating pool with 8 workers, operating on rows

Time elapsed (8 processes): 112.376024 seconds
Nonzero count after solve (should be >> 0): 295464


In [ ]:
with open(OUT_TXT, 'a') as f:
    f.write(f'Creating pool with {NUM_PROCESSES} workers, operating on rows\n\n')
    f.write(f'Time elapsed ({NUM_PROCESSES} processes): {end_time - start_time:.6f} seconds\n')
    f.write(f'Nonzero count after solve (should be >> 0): {np.count_nonzero(predictions)}\n\n')
    
    f.write(f'Max error: {errors.max()}\n')
    f.write(f'Mean error (including zeros): {errors.mean()}\n')
    f.write(f'Mean error (excluding zeros): {np.mean(errors[np.nonzero(errors)])}\n')
    
print(f'Max error: {errors.max()}')
print(f'Mean error (including zeros): {errors.mean()}')
print(f'Mean error (excluding zeros): {np.mean(errors[np.nonzero(errors)])}')

Max error: 1.4096207023510205
Mean error (including zeros): 0.006328477096053895
Mean error (excluding zeros): 0.026026360953028654


In [7]:
# -----------------------------------------------------------------
# save results
# -----------------------------------------------------------------
predictionsT = np.transpose(predictions, (1, 0, 2))
errorsT = np.transpose(errors)

if (bsdf == 'diffuse'):
    predictedRGB = predictions
    predictionsImage = Image.fromarray((predictedRGB * 255).astype(np.uint8))
    predictionsImage.save(IMAGE_PREDICTIONS_RGB)

if (bsdf == 'bsdf'):
    print(predictions.size)
    predictedRGB = predictionsT[:, :, :3]
    predictedRoughness = predictionsT[:, :, 3]
    predictedMetallic = predictionsT[:, :, 4]
    
    predictionsImage = Image.fromarray((predictedRGB * 255).astype(np.uint8))
    predictionsImage.save(IMAGE_PREDICTIONS_RGB)
    
    predictionsImage = Image.fromarray((predictedRoughness * 255).astype(np.uint8))
    predictionsImage.save(IMAGE_PREDICTIONS_ROUGHNESS)
    
    predictionsImage = Image.fromarray((predictedMetallic * 255).astype(np.uint8))
    predictionsImage.save(IMAGE_PREDICTIONS_METALLIC)

# error image
errorsImage = Image.fromarray((errors/errors.max() * 255).astype(np.uint8))
errorsImage.save(IMAGE_ERRORS)

# gamma correction: https://gist.github.com/rkdgusrn1212/5eb95c0c019e280f07269967017c4f38
im = Image.open(IMAGE_PREDICTIONS_RGB)

row = im.size[0]
col = im.size[1]
resultImage = Image.new("RGB", (row, col))

for x in range(1 , row):
    for y in range(1, col):
        r = min(255, pow(im.getpixel((x,y))[0]/255, (1/GAMMA))*255)
        g = min(255, pow(im.getpixel((x,y))[1]/255, (1/GAMMA))*255)
        b = min(255, pow(im.getpixel((x,y))[2]/255, (1/GAMMA))*255)
        resultImage.putpixel((x,y), (int(r), int(g), int(b)))
            
resultImage.save(IMAGE_RGB_GAMMA)

if (bsdf == 'bsdf'):
    images = [(IMAGE_PREDICTIONS_METALLIC, IMAGE_METAL_GAMMA),
              (IMAGE_PREDICTIONS_ROUGHNESS, IMAGE_ROUGH_GAMMA)]
    
    for imageName, outImage in images:
        im = Image.open(imageName)
        resultImage = Image.new("L", (row, col))

        for x in range(1 , row):
            for y in range(1, col):
                r = min(255, pow(im.getpixel((x,y))/255, (1/GAMMA))*255)
                resultImage.putpixel((x,y), int(r))
            
        resultImage.save(outImage)

1310720
